In [1]:
import json
import base64
import hashlib
from urllib.parse import urlparse
from base64 import b64encode
from haralyzer import HarParser

In [2]:
host = "faz.net"
type = "news/desktop"
path = "../archives/" + type + "/www." + host + ".har"

# Read the content of the HAR file and convert it to a dictionary
with open(path, "r", encoding="utf-8") as har_file:
    harData = json.load(har_file)

parser = HarParser(harData)
entries = parser.har_data['entries']

print("Number of entries: {}".format(len(entries)))

Number of entries: 2132


In [3]:
email = 'yannick.nastja@gmail.com'

def contains_md5(text, host, location):
    hashed_email = hashlib.md5(email.encode()).hexdigest()
    if hashed_email in text:
        print(f"MD5 hash {hashed_email} found in {location} {host}: {text}")

def contains_base64(text, host, location):
    encoded_email = base64.b64encode(email.encode()).decode()
    if encoded_email in text:
        print(f"Base64-encoded email address found in {location} {host}: {text}")

def contains_sha256(text, host, location):
    hashed_email = hashlib.sha256(email.encode()).hexdigest()
    if hashed_email in text:
        print(f"SHA256 hash {hashed_email} found in {location} {host}: {text}")

def contains_sha1(text, host, location):
    hashed_email = hashlib.sha1(email.encode()).hexdigest()
    if hashed_email in text:
        print(f"SHA1 hash {hashed_email} found in {location} {host}: {text}")

def contains_sha224(text, host, location):
    hashed_email = hashlib.sha224(email.encode()).hexdigest()
    if hashed_email in text:
        print(f"SHA224 hash {hashed_email} found in {location} {host}: {text}")

def contains_sha3_512(text, host, location):
    hashed_email = hashlib.sha3_512(email.encode()).hexdigest()
    if hashed_email in text:
        print(f"SHA3_512 hash {hashed_email} found in {location} {host}: {text}")

def contains_mail(text, host, location):
    if email in text:
        print(f"Mail found in {location} {host}")

def findMail(entries):
    for entry in entries:
        request = entry.get('request', {})
        response = entry.get('response', {})
        host = urlparse(request.get('url', '')).hostname

        for text in [request.get('url', ''), str(request.get('headers', '')), str(request.get('postData', ''))]:
            contains_md5(text, host, "REQUEST")
            contains_base64(text, host, "REQUEST")
            contains_sha256(text, host, "REQUEST")
            contains_mail(text, host, "REQUEST")
            contains_sha1(text, host, "REQUEST")
            contains_sha224(text, host, "REQUEST")
            contains_sha3_512(text, host, "REQUEST")

        for text in [str(response.get('content', {}).get('text', ''))]:
            contains_md5(text, host, "RESPONSE")
            contains_base64(text, host, "RESPONSE")
            contains_sha256(text, host, "RESPONSE")
            contains_mail(text, host, "RESPONSE")
            contains_sha1(text, host, "RESPONSE")
            contains_sha224(text, host, "RESPONSE")
            contains_sha3_512(text, host, "RESPONSE")




In [4]:
newsHosts = ["zeit.de", "cnn.com", "faz.net", "merkur.de", "n-tv.de", "sueddeutsche.de", "telegraph.co.uk", "theguardian.com", "thehindu.com", "zeit.de"]
types = ["news/desktop/www.", "news/mobile/m."]

for host in newsHosts:
    for type in types:
        print("Host: " + host + " (" + type.rsplit("/")[1] + ")")
        path = "../archives/" + type + host + ".har"

        # Read the content of the HAR file and convert it to a dictionary
        with open(path, "r", encoding="utf-8") as har_file:
            harData = json.load(har_file)

        parser = HarParser(harData)
        entries = parser.har_data['entries']

        print("Number of entries: {}".format(len(entries)))
        findMail(entries)

    print("\n")

Host: zeit.de (desktop)
Number of entries: 729
Mail found in RESPONSE www.zeit.de
Host: zeit.de (mobile)
Number of entries: 617
Mail found in RESPONSE www.zeit.de
Mail found in RESPONSE www.zeit.de


Host: cnn.com (desktop)
Number of entries: 773
Mail found in REQUEST audience.cnn.com
Mail found in REQUEST audience.cnn.com
Mail found in RESPONSE audience.cnn.com
Mail found in RESPONSE audience.cnn.com
Host: cnn.com (mobile)
Number of entries: 1168
Mail found in RESPONSE audience.cnn.com
Mail found in RESPONSE audience.cnn.com
Mail found in RESPONSE audience.cnn.com
Mail found in REQUEST audience.cnn.com


Host: faz.net (desktop)
Number of entries: 2132
SHA256 hash 69638e8bc82beb01fbd95ae63968c186965aa5e18a4fd0b343639857d05be7c9 found in REQUEST www.facebook.com: https://www.facebook.com/tr/?id=494203564082803&ev=SubscribedButtonClick&dl=https%3A%2F%2Fwww.faz.net%2Fmein-faz-net%2F%3FredirectUrl%3D%252Faktuell%252F&rl=https%3A%2F%2Fwww.faz.net%2Fmein-faz-net%2F%3Fot%3Dde.faz.ot.body-vm%2